# Práctica 2: Búsquedas en espacios de estados. Búsquedas Informadas.
* versión 1.0 (18 Febrero 25)

<center><h3>
    Asier Acuña Casal
</h3></center>

## Instrucciones

Esto es **Jupyter Notebook**, un documento que integra código Python en un archivo Markdown.
Esto nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica.

Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**

Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión, responde a todas ellas en la sección reservada para ese fin: **"Respuestas a los cuestionarios"**. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

Recuerda que esta práctica está planificada para **2 sesiones** (Semanas 4 y 5) y a lo largo de todas ellas se discutirán grupalmente parte de las preguntas y problemas. 

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELLIDOS_NOMBRE_Busquedas_Informadas.zip` que contenga los seguientes ficheros:

 * `APELLIDOS_NOMBRE_Busqueda_Informada.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELLIDOS_NOMBRE_Busqueda_Informada.ipynb`: Archivo fuente Jupyter Notebook.
 * Resto de archivos de códigos y datos que sean necesario para reejecutar tu práctica.


# Búsquedas informadas

El [capítulo 4](http://aima.cs.berkeley.edu/4th-ed/pdfs/newchap04.pdf) del libro de referencia AIMA cubre el tema de las búsquedas informadas. Además de haber visto los algoritmos de búsqueda ciega en espacios de estados, también habéis visto algoritmos de búsqueda heurística, que en contraposición a los algoritmos de búsqueda ciega, incorporan una función heurística y de evaluación que permite realizar una búsqueda más guiada (por tanto, menos ciega) teniendo en cuenta información potencialmente útil. A la búsqueda heurística se le conoce también como búsqueda informada, por incorporar esa información de guiado adicional en su función. 


Como cabría esperar, un buen número de estrategias informadas también vienen implementadas en AIMA-Python. Lo interesante de disponer de la implementación de varios algoritmos en una librería/módulo es que podemos disponer de ellos sin demasiado esfuerzo, pudiendo probarlos y realizar comparativas para estudiar su desempeño.


En esta práctica vamos a profundizar en la implementación de estrategias de búsqueda informadas. De forma similar a como hiciste en la anterior práctica, que se limitaba a las estrategias a ciegas, puedes importar los módulos de la práctica de la forma habitual.

In [ ]:
# contiene algoritmos de búsqueda implementado en AIMA
from search_mod import *

# algunas funciones auxiliares
from helpers_mod import *

Vamos a definir de nuevo la clase EightPuzzle como subclase de Problem como ya se hizo en la práctica anterior. Sin embargo, en esta ocasión, fíjate que el método h sí que tiene implementación.

In [ ]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[node.state.index(num)]-X[self.goal.index(num)]) + abs(Y[node.state.index(num)]-Y[self.goal.index(num)]) for num in range(1, 9))
        
     # Esta es una posible implementaciónde la distancia de Gaschnig
    def h3(self, node):        
        node_state = list(node.state)
        goal = list(self.goal)
        distance = 0
        while node_state != goal:            
            if (node_state.index(0) == goal.index (0)):                
                idxGap = node_state.index(0)
                swapIdx = 0                
                misplacedFound = False
                while (swapIdx < 9 and not misplacedFound):
                    if node_state[swapIdx] != goal[swapIdx] and node_state[swapIdx] != 0:
                        misplacedFound = True                        
                    else:
                        swapIdx = swapIdx + 1

                swapValue = node_state[swapIdx]
                node_state [idxGap] = swapValue
                node_state [swapIdx] = 0
                
            else:                            
                idxGap = node_state.index(0)
                
                swapValue = goal[idxGap]                
                swapIdx = node_state.index(swapValue)

                node_state[idxGap] = swapValue
                node_state[swapIdx] = 0                         
                
                
            distance = distance + 1

        return distance
    
    def h(self, node): return self.h2(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))


def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')



Volvamos a crear instancias de problemas del 8-Puzzle:

In [ ]:
# Ejemplos de creación de instancias del problema del 8-puzzle
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))


y a continuación podrías hacer uso cualquiera de las estrategias de búsqueda disponibles en la librería para resolver el problema. Algunas de las estrategias de búsqueda, tanto ciegas como heurísticas, que podemos utilizar por venir ya implementadas son las siguientes:

|Algoritmo|Función python|*f*|
|:---------|--|---:|
|Breadth search |`breadth_first_search`| no-informada |
|Depth search | `depth_first_search`|no-informada|
|Limited Depth search | `depth_limited_search`|no-informada|
|Breadth best-first search |`breadth_first_bfs`| *f = len*|
|Depth best-first search |`depth_first_bfs`| *f = -len*|
|Greedy best-first search |` greedy_bfs `| *f = h*|
|Uniform-cost search |` uniform_cost_search `| *f = g*|
|A* search |` astar_search `| *f = g + h*|
|Weighted A* search | ` weighted_astar_search ` | *f = g + 1.4 &times; h*|


In [ ]:
# resolver una instancia particular con una estrategia de búsqueda concreta
# retorna la solución, como una instancia de Node
print(f'Resolver el problema {e1} con Búsqueda en Anchura')
sol = breadth_first_search(e1)
print ("Nodo solución: ", sol)
print (path_states(sol))

print(f'Resolver el problema {e1} con Búsqueda de coste uniforme')
sol = uniform_cost_search(e1)
print ("Nodo solución: ", sol)
print (path_states(sol))

print(f'Resolver el problema {e1} con Búsqueda A* (con una de la heurísticas de 8-Puzzle)')
sol = astar_search(e1)
print ("Nodo solución: ", sol)
print (path_states(sol))


Recuerda que puedes hacer uso de funciones como `path_states()` y `path_actions()` para obtener la cadena de estados que llevan a la solución y las acciones respectivas, pudiendo observar los estados que se alcanzan en el camino de la solución y las acciones que se aplicaron para ello. 

❓ **Pregunta 1**. Ejecuta la comparativa de todas las estrategias búsqueda utilizando la función `report` y responde a las siguientes cuestiones de manera justificada en relación a las medidas obtenidas.

- 1.1) ¿Qué desventajas o limitaciones consideras que tienen las búsquedas no-informadas? ¿Por qué? 
**Pista**: Revisa lo que viste en la expositiva al respecto de la complejidad computacional.

- 1.2) ¿Qué estrategias obtienen soluciones óptimas? ¿Si cambiaramos las instancias del problema, esas estrategias seguirían obteniendo soluciones óptimas siempre?

- 1.3) Discute de manera justificada el rendimiento de los algoritmos en términos del coste del camino/solución, el número de nodos explorados, y si obtiene la solución óptima.

- 1.4) ¿Qué (función) heurística ha utilizado `astar_search`? Simplemente indica cuál es pues ya entrarás en detalles más adelante.

**Pistas**: En la expositiva has visto lo que es una búsqueda óptima, que es aquella que encuentra la mejor solución, si existe, independientemente de si el proceso de búsqueda es eficiente o no. Por lo tanto, aquí nos referimos a la solución óptima como aquella mejor solución posible.


# EJERCICIO 1

In [ ]:
busquedas = (
    breadth_first_search,
    depth_limited_search,
    breadth_first_bfs,
    greedy_bfs,
    uniform_cost_search,
    astar_search,
    weighted_astar_search
    )

ejemplos = (e1, e2, e3,e4, e5)
report(busquedas, ejemplos)

## 1.1) 
A simple vista podemos observar que el mayor problema es la baja eficiencia, debido a la gran generación de nodos que provoca una gran complejidad espacial, además de la complejidad computacional. 

## 1.2) 
Menos la de búsqueda en profundidad, todas las demás.

## 1.3)
- **BFS**: es una estrategia óptima, ya que siempre va a encontrar una solución si existe, sin embargo genera muchos nodos, de esta forma su complejidad espacial es muy alta. Por otra parte, su complejidad temporal es exponencial, siendo, además, muy lento.
  
- **DLS**: en este caso genera muchos menos nodos, pero esto puede estar provocado por el límite de búsqueda que no le permite avanzar. Además, no es óptimo, puesto que no siempre encuentra solución. De esta forma su complejidad espacial es baja (b*m). La complejidad temporal es igual que en anchura.

- **BF-BFS**: este tipo de búsqueda, a pesar de ser informada, genera muchos nodos  (incluso más que la BFS normal), teniendo una complejidad espacial muy alta. De esta forma, al ser una BFS es completa pero muy poco eficiente.

- **GBFS**: la búsqueda greedy genera muchos menos nodos (complejidad espacial baja), sin embargo recorre más que el resto de búsqueda, por lo que puede ser poco eficiente. Al ser BFS sigue siendo completa.

- **UCS**: esta búsqueda es completa y óptima, es una aplicación de BFS. Genera más nodos que el BFS normal, concretamente, los mismos que BFBFS. Sin embargo, la complejidad computacional es ligeramente menor que la búsqueda en profundidad.

- **As**: la búsqueda A* recorre considerablemente menos nodos que el resto de búsquedas en anchura, sin embargo recorre exactamente el mismo número de ellos. La complejidad temporal depende de la heurística. 

- **WAs**: esta búsqueda ponderada A* es la búsqueda completa que menos nodos genera, a pesar de que acaba recorriendo los mismos que el resto. Por eso podemos afirmar que es eficiente en cuanto a la complejidad espacial.

## 1.4)
Usa una función de costo f = g(x) + h(x), siendo la función heurística h(x) y x el estado actual.

## Implementación de heurísticas para A*: problema del 8-puzzle



Las estrategias de búsquedas informadas (heurísticas) utilizan conocimiento específico del problema
y vienen dirigidas por una función de evaluación que gobierna la expansión de los nodos de la forma
f(n) = g(n) + h(n), donde 

- g(n) = el coste real de haber alcanzado el nodo/estado n

- h(n) = el coste estimado del camino más "barato" desde el estado n hasta un estado objetivo (Nota: el más "barato" se suele referir al más corto o al de menor coste, pero depende del problema específico)

y que, por tanto, da cuenta del coste del camino construido hasta el estado actual y el coste estimado
heurísticamente de lo que queda hasta alcanzar un estado objetivo. La función heurística h(n) es la
responsable de codificar el conocimiento específico para realizar la búsqueda de manera informada,
y es considerada una función arbitraria, no-negativa, que cumple además la restricción de h(n)=0 si n
es un estado objetivo.

De esta forma, existen algunas estrategias de búsqueda bien conocidas caracterizadas por la
estructura de la función de evaluación, que ya has empleado anteriormente.
- Búsqueda voraz primero el mejor (Greedy best-first search): intenta expandir el nodo que está más cerca del objetivo, con el fundamento de que esto lleve probablemente a una solución más rápidamente a expensas de la optimalidad de la solución. La evaluación se lleva a cabo sólo con la función heurística: f(n) = h(n)
- Búsqueda A*: De manera amplia se combina tanto el coste del camino recorrido como la estimación de lo que queda, con el objetivo de minimizar el coste total estimado mientras considera reducir el coste de la solución intermedia construida: f(n) = g(n) + h(n)

Puedes comprobar la función f de cada una de estas estrategias, como sigue.

In [ ]:
psource (greedy_bfs)
psource (astar_search)
psource (best_first_search)
psource (g)

Típicamente, se pueden diseñar funciones heurísticas h transformando el problema original en un
problema relajado o problema con restricciones relajadas. Por ejemplo, el problema original del 8-
puzzle se define como sigue: *Una ficha puede moverse de la posición A a la B si A es adyacente (ya sea
horizontal o verticalmente) a B, y B es el blanco*. Partiendo de esa formulación, podemos generar tres
problemas relajados mediante la eliminación de una o ambas condiciones:
- (a) Una ficha puede moverse de la posición A a la B is A es adyacente a B.
- (b) Una ficha puede moverse de la posición A a la B si B es blanco.
- (c) Una ficha puede mover de la posición A a la B.

En los ejemplos de la implementación del 8-puzzle, las funciones de evaluación emplean como función heurística
h las siguientes, que se corresponden con la formulación de los problemas relajados (c), (a) y (b) respectivamente:
- h1 (n) = número de fichas que en el estado n están descolocadas o fuera de su posición en el objetivo = distancia de Hamming
- h2 (n) = suma de las distancias ortogonales de cada ficha a su posición en el estado objetivo = distancia de Manhattan
- h3 (n) = número de intercambios con el blanco para transformar el estado inicial en el objetivo = distancia de Gaschnig


Cabe destacar que, por defecto, la implementación `EightPuzzle` utiliza la distancia de Manhattan (i.e., h2). Por tanto, para hacer uso de la h1 y poder incluir el uso de dicha estrategia en la comparativa necesitamos definir previamente una función `astar_misplaced_tiles` como se ejemplifica a continuación.

In [ ]:
def astar_misplaced_tiles(problem): return astar_search(problem, h=problem.h1)
def astar_gaschnig(problem): return astar_search(problem, h=problem.h3)


Que podemos entonces utilizar como cualquier otra función que implementa una estrategia de búsqueda.

In [ ]:
sol2=astar_misplaced_tiles(e1)
for s in path_states(sol2):
    print(board8(s))

report([astar_misplaced_tiles, astar_search], [e1, e3])

❓ **Pregunta 2**. Compara la estrategia A* con las tres heurísticas del 8-Puzzle y discútelas de manera justificada. Incluye la búsqueda en anchura en la comparativa. ¿Qué función heurística es más informada? ¿Cómo ha funcionado en promedio la tercera heurística en relación con las otras dos? ¿En qué proporción o orden de magnitud funciona la búsqueda con heurística que sin ella?

No olvides incluir el código que has tenido que escribir para realizar la tabla comparativa resultante. 

**Pista**: Cuando disponemos de varias funciones heurísticas que utilizan la estrategia A\*, decimos que una función es más informada que otra cuando la primera necesita expandir menos nodos que la segunda. Asumimos entonces que debido a la mayor información codificada en la función heurística, o al menos aprovechada de mejor forma, permite guiar mejor el proceso de búsqueda de tal forma que la estimación dada por la función *h* se ajusta/acerca más a la realidad del coste pendiente.


# EJERCICIO 2

In [ ]:
funciones = (
    astar_search(p, h=h1),
    astar_search(p, h=h2),
    astar_search(p, h=h3)
)

problema = (e1, e2, e3, e4, e5)

for f in funciones:
    for p in problema:
        report(breadth_first_search(p), p)

: 

EWOJFOSÑDFMARJROSJ


# El problema de Viajante de Comercio (VC) y Búsqueda A*

Recupera el código de tu implementación de `ProblemaViajanteComercio(Problem)` que realizaste en la anterior práctica, e incorpórala en la siguiente celda. Fíjate que esa implementación tenía el método h para devolver 0, que era entonces suficiente puesto que hasta el momento solo estabas utilizando estrategias de búsqueda a ciegas. 


In [ ]:
# Celda asociada
# Completa la implementación

class ProblemaViajanteComercio(Problem):
    """ Problema Viajante de Comercio
    """    
            
    def actions(self, state):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""  
        # ToDo
        return []
    
    def result(self, state, action):
        """produce el nuevo estado al anadir la accion seleccionada"""                                      
        # ToDo        
        return state
    
    def is_goal(self, state):        
        # ToDo
        return False
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""        
        # ToDo
        return 0   
    
    def h(self, node): 
        # ToDo
        return 0


### Implementación de heurísticas para VC

La elección de heurísticas que estimen el coste de alcanzar un estado objetivo a partir de un estado dado es crucial para que la estrategia de búsqueda A* sea guiada con mayor eficacia. Recordemos que cuanta más precisa sea la estimación que efectue la función h (i.e., más cercana al coste real), más informada será la búsqueda. Es conveniente que la heurística subestime el coste real para asegurar que la estrategia sigue siendo admisible, y por tanto no descarte estados que puedan conducir a la solución óptima.


❓ **Pregunta 3**. Realiza las implementaciones necesarias para incorporar heurísticas a la implementación A* del VC y responde a las siguientes cuestiones.

- 3.1) Diseña e implementa las siguientes funciones heurísticas que se proponen para una estrategia de búsqueda A* explicando brevemente cómo las has implementado:
    - h1 (n) = vecino más próximo: esta heurística considera como coste estimado la distancia al vecino más próximo que no haya sido visitado todavía, persiguiendo de esta forma que se produzca el mínimo incremento en longitud al añadir la ciudad al camino.
    - h2 (n) = vecino más lejano: esta heurística considera como coste estimado la distancia al vecino más lejano que no haya sido visitado todavía, teniendo en cuenta de esta forma posible coste en que se podría incurrir luego por esos nodos más lejanos pendientes.
    - h3 (n) = distancias medias: esta heurística considera realizar una estimación calculada como la distancia media a los nodos pendientes de visitar.
- 3.2) Utiliza el grafo de las 8 ciudades gallegas como datos de entrada para comprobar la correctitud de tu implementación. ¿Cuál es la distancia obtenida en la solución óptima? Muestra el resultado de la ejecución A* con las diferentes heurísticas implementadas.
- 3.3) Para cargar varias instancias del problema como entrada, utiliza algunos ficheros del directorio `./data`, y lleva a cabo la comparación de las heurísticas implementadas. Explica y justifica los resultados obtenidos. 

**Nota:** Ten en cuenta que para extraer conclusiones científicamente válidas puede no ser suficiente utilizar una única instancia del problema como entrada.



In [ ]:
# Celda asociada a las Preguntas 3.1, 3.2 y 3.3



❓ **Pregunta 4:** Diseña e implementa otra función heurística para resolver el problema del VC con A*. En particular, tal y como sugiere el libro de Russell & Norvig, se puede utilizar una función heurística basada en el árbol de expansión mínima (minimum spanning tree - MST) como subestimación del coste, que se podría obtener mediante los algoritmos de Prim o Kruskal que has visto en una asignatura previa. Estudia su rendimiento de esta nueva heurística en relación a las otras funciones heurísticas que implementaste en la pregunta 3. 



In [ ]:
# Incluye aquí la resolución, estudio y discusión de resultados
# emplea las celdas que sean necesarias para que esté bien organizado y sea legible


**************************************************************

# Respuestas a los ejercicios y evaluación

**RECUERDA:** No te olvides de escribir tu nombre y apellidos en la segunda celda del presente documento.

Incluye las celdas que consideres oportunas para que sea legible. Incluye la formulación de las preguntas para una mejor comprensión y facilitar su corrección.

La práctica se evaluará siguiendo un esquema 8+2, en el que los primeros 8 puntos se evaluarán mediante un cuestionario automático de evaluación que consideran las secciones de la práctica relativa a las preguntas 1, 2 y 3, mientras que los últimos 2 puntos se corresponderán al ejercicio planteado en la Pregunta 4.

El cuestionario automático de evaluación se realiza normalmente en la primera sesión de la próxima práctica, y se centrará en la resolución por tu parte de diversas cuestiones prácticas relacionadas tanto con los aspectos conceptuales como de implementación de tu práctica, pudiendo ser necesaria la ejecución, adaptación y modificación de la misma durante la duración de la prueba.

Independientemente del cuestionario automático de evaluación, las preguntas planteadas deben responderse en el notebook de manera clara y precisa. Esas preguntas generales están diseñadas para formarte, y te servirán para razonar y reflexionar sobre el tema, así como también para fomentar una discusión constructiva con los docentes durante las sesiones. No se dará por superada la práctica si las respuestas en el notebook no son adecuadas. 

Recuerda que el plagio/copia completa o parcial de prácticas de otros compañeros acarrea no solo el suspenso de la práctica en curso sino también de la asignatura en dos convocatorias.

**************************************************************